In [139]:
#importazione librerie
import pandas as pd
import re
import numpy as np

In [ ]:
times=pd.read_csv('C:/Users/Antonino Guidotto/Desktop/2020/timesData.csv') #lettura del file .csv
times.head() #mostriamo le prime 5 righe del dataset 

# 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [ ]:
#attraverso la funzione groupby abbiamo raggruppato il dataset per il nome dell'università, 
#individuando grazie alle funzioni idxmax e idxmin, l'indice che possedeva il valore minimo e massimo dell'attributo anno
#dopo, grazie alla funzione .loc è stata istratta l'intera riga contenente il risultato delle operazioni descritte

times_max=times.loc[times.groupby('university_name').idxmax()['year']]  
times_min=times.loc[times.groupby('university_name').idxmin()['year']]

#attraverso la funzione reset_index, è stato resettato l'indice in modo da farlo ripartire da zero.
times_max.reset_index(level=0,inplace=True)
times_min.reset_index(level=0,inplace=True)

times_max.head() 

In [ ]:
times_min.head()

# 2. For each university, compute the improvement in income between the least recent and the most recent data point

In [ ]:
df=times.copy() #viene creata una copia del dataframe times in modo da non intaccare eventuali esecuzioni successive

#iteriamo su tutte le righe del dataframe df, e se nella colonna relativa all'income, il record presenta il carattere '-'
# allora la riga viene eliminata
for index, row in df.iterrows():
    if row['income']=='-':
        df.drop([index],inplace=True)


df_max=df.loc[df.groupby('university_name').idxmax()['year']]
df_min=df.loc[df.groupby('university_name').idxmin()['year']]
  
df_max.reset_index(level=0,inplace=True)
df_max.head()

In [ ]:
df_min.reset_index(level=0,inplace=True)
df_min.head()

In [ ]:
# viene fatta una merge tra i due dataframe sull'attributo 'university_name'
fuso = pd.merge(df_min, df_max, on= 'university_name', right_index = True)
#vengono rinominate le colonne derivanti dall'operazione di merge
fuso.rename(columns={'income_x':'income_min'}, inplace=True)
fuso.rename(columns={'income_y':'income_max'}, inplace=True)
fuso.rename(columns={'year_x':'year_min'}, inplace=True)
fuso.rename(columns={'year_y':'year_max'}, inplace=True)

# viene convertito il tipo dell'attributo in float
fuso['income_min'] = fuso['income_min'].astype(float)
fuso['income_max'] = fuso['income_max'].astype(float)

#creazione di un nuovo dataframe contenente le colonne utili per questa elaborazione
fuso_income=fuso[['university_name','income_min','year_min', 'income_max','year_max']]

fuso_income.head()

In [ ]:
# viene calcolata la differenza tra il valore di 'income_max' e il valore di 'income_min'
fuso_income['improvement']=fuso_income['income_max']-fuso_income['income_min']

fuso_income.head()


# 3. Find the university with the largest increase computed in the previous point

In [ ]:
# si usa la funzione .iloc per estrarre la riga con le caratteristiche desiderate grazie alla funzione .idxmax
fuso_income.iloc[fuso_income['improvement'].idxmax()]

# 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [ ]:
shanghai=pd.read_csv('C:/Users/Antonino Guidotto/Desktop/2020/shanghaiData.csv')

shanghai.head()

In [ ]:
#creazione di un dataframe "ribaltato", al fine di avereun dataframe dove l'ordine cronologico non sia dal data point
# meno recente al più recente, ma viceversa.
ribaltato_s=shanghai.iloc[::-1]

ribaltato_s.head()

In [ ]:
cwur=pd.read_csv('C:/Users/Antonino Guidotto/Desktop/2020/cwurData.csv')
cwur.head()

In [ ]:
ribaltato_c=cwur.iloc[::-1]
ribaltato_c.head()

In [ ]:
#creazione di un dizionario
diz_max_rank_times={}

i=0

# iterazione sul dataframe times_max
for uni in times_max['university_name']:
    ranking=times_max['world_rank'][i] #estrazione del valore dell'attributo 'ranking' associato all'università in questione
    anno=times_max['year'][i] #estrazione del valore dell'attributo 'anno' associato all'università in questione
    if uni not in diz_max_rank_times: #condizione, se l'università non si trova già nel dizionario...
        # se il trattino si trova nel ranking, allora la posizione nel ranking sarà data dal valore medio tra i due estremi del range
        if '-' in ranking: 
            num1=int(ranking[:3]) # primo valore del range
            num2=int(ranking[4:]) # secondo valore del range
            val_centrale=(num2+num1)/2 # calcolo del valore centrale (media aritmetica)
            #inserimento della coppia chiave:valori nel dizionario, con chiave 'uni' e valori 'ranking' e 'anno'
            diz_max_rank_times[uni]=(val_centrale, anno) 
            
        else:
            diz_max_rank_times[uni]=(ranking, anno)
            
    
    i+=1

print(diz_max_rank_times)

In [ ]:
#trasformazione del dizionario in dataframe
df_max_rank_times=pd.DataFrame.from_dict(diz_max_rank_times, orient='index',columns=['world_rank','year'])

df_max_rank_times.reset_index(level=0,inplace=True)
df_max_rank_times.rename(columns={'index' : 'university_name'}, inplace=True)
df_max_rank_times.head()


In [ ]:
diz_max_rank_shanghai={}
i=len(ribaltato_s)-1
for uni in ribaltato_s['university_name']:
    ranking=ribaltato_s['world_rank'][i]
    anno=ribaltato_s['year'][i]
    if uni not in diz_max_rank_shanghai:
        if '-' in ranking:
            num1=int(ranking[:3])
            num2=int(ranking[4:])
            val_centrale=(num2+num1)/2
            diz_max_rank_shanghai[uni]=(val_centrale, anno)
            
        else:
            diz_max_rank_shanghai[uni]=(ranking, anno)
            
    
    i-=1

print(diz_max_rank_shanghai)

In [ ]:
df_max_rank_shanghai=pd.DataFrame.from_dict(diz_max_rank_shanghai, orient='index',columns=['world_rank','year'])
df_max_rank_shanghai.reset_index(level=0,inplace=True)
df_max_rank_shanghai.rename(columns={'index' : 'university_name'}, inplace=True)
df_max_rank_shanghai.head()

In [ ]:
diz_max_rank_cwur={}
i=len(ribaltato_c)-1
for uni in ribaltato_c['institution']:
    ranking=ribaltato_c['world_rank'][i]
    anno=ribaltato_c['year'][i]
    if uni not in diz_max_rank_cwur:
        diz_max_rank_cwur[uni]=(ranking, anno)
            
    
    i-=1

print(diz_max_rank_cwur)

In [ ]:
df_max_rank_cwur=pd.DataFrame.from_dict(diz_max_rank_cwur, orient='index',columns=['world_rank','year'])
df_max_rank_cwur.reset_index(level=0,inplace=True)
df_max_rank_cwur.rename(columns={'index' : 'university_name'}, inplace=True)
df_max_rank_cwur.head()

In [ ]:
# operazione di merge tra i 3 dataframe sull'attributo 'university_name'
fuso_parziale = pd.merge(df_max_rank_times, df_max_rank_shanghai, on= 'university_name', right_index = True)
fuso_ranking=pd.merge(fuso_parziale, df_max_rank_cwur, on= 'university_name', right_index = True)

fuso_ranking.rename(columns={'world_rank_x' : 'world_rank_times'}, inplace=True)
fuso_ranking.rename(columns={'year_x' : 'year_times'}, inplace=True)
fuso_ranking.rename(columns={'world_rank_y' : 'world_rank_shanghai'}, inplace=True)
fuso_ranking.rename(columns={'year_y' : 'year_shanghai'}, inplace=True)
fuso_ranking.rename(columns={'world_rank' : 'world_rank_cwur'}, inplace=True)
fuso_ranking.rename(columns={'year' : 'year_cwur'}, inplace=True)
fuso_ranking.reset_index(drop=True, inplace=True)

fuso_ranking.head()

In [ ]:
li=[] #creazione di una lista vuora

for elem in fuso_ranking['world_rank_times']:
    if '=' in str(elem):
        # eliminazione del carattere '=' dalla colonna indicata si usa  la funzione .replace
        elem_nuovo=float(elem.replace('=','')) 
        li.append(elem_nuovo)
    else: 
        li.append(float(elem))

fuso_ranking['world_rank_times']=li # sostituzione della colonna con la lista 'li'


In [ ]:
i=0
#creazione di 3 liste vuote
li1=[]
li2=[]
li3=[]

# vengono fatti dei confronti tra le posizioni delle università nei vari ranking al fine di calcolare le differenze.
while i<len(fuso_ranking):
    if fuso_ranking['year_times'][i]>=fuso_ranking['year_shanghai'][i]:
        diff_t_s=float(fuso_ranking['world_rank_times'][i])-float(fuso_ranking['world_rank_shanghai'][i])
        li1.append(diff_t_s) #la funzione .append, permette di aggiungere elementi ad una lista
        
    else:
        diff_s_t=float(fuso_ranking['world_rank_shanghai'][i])-float(fuso_ranking['world_rank_times'][i])
        li1.append(diff_s_t)
        
    if fuso_ranking['year_times'][i]>=fuso_ranking['year_cwur'][i]:
        diff_t_c=float(fuso_ranking['world_rank_times'][i])-float(fuso_ranking['world_rank_cwur'][i])
        li2.append(diff_t_c)
    else:
        diff_c_t=float(fuso_ranking['world_rank_cwur'][i])-float(fuso_ranking['world_rank_times'][i])
        li2.append(diff_c_t)
        
    if fuso_ranking['year_shanghai'][i]>=fuso_ranking['year_cwur'][i]:
        diff_s_c=float(fuso_ranking['world_rank_shanghai'][i])-float(fuso_ranking['world_rank_cwur'][i])
        diff_s_t=float(fuso_ranking['world_rank_shanghai'][i])-float(fuso_ranking['world_rank_times'][i])
        li3.append(diff_s_t)
    else:
        diff_c_s=float(fuso_ranking['world_rank_cwur'][i])-float(fuso_ranking['world_rank_shanghai'][i])
        li3.append(diff_c_s)
    
    i+=1
    
fuso_ranking['diff_t_s']=li1
fuso_ranking['diff_t_c']=li2
fuso_ranking['diff_s_c']=li3

In [ ]:
fuso_ranking.head()

In [ ]:
uni=[]
massimo=[]
i=0
# grazie a questa serie di ulteriori confronti tra le differenze trovate al punto sopra, 
#siamo in grado di individuare la differenza massima
while i<len(fuso_ranking):
    if fuso_ranking['diff_t_s'][i]>=fuso_ranking['diff_t_c'][i]:
        if fuso_ranking['diff_t_s'][i]>=fuso_ranking['diff_s_c'][i]:
            uni.append(fuso_ranking['university_name'][i])
            massimo.append(fuso_ranking['diff_t_s'][i])
        else:
            uni.append(fuso_ranking['university_name'][i])
            massimo.append(fuso_ranking['diff_s_c'][i])
            
    elif fuso_ranking['diff_t_c'][i]>=fuso_ranking['diff_s_c'][i]:
        
        uni.append(fuso_ranking['university_name'][i])
        massimo.append(fuso_ranking['diff_t_c'][i])
    else:
        uni.append(fuso_ranking['university_name'][i])
        massimo.append(fuso_ranking['diff_s_c'][i])
        
    i+=1
    
        
fuso_ranking['max_diff']=massimo
fuso_ranking.head()
            
       


## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country

In [ ]:
times_max

In [ ]:
times_max['num_students']=times_max['num_students'].str.replace(',','')
times_max['num_students']=times_max['num_students'].astype('float')

In [ ]:
#viene fatta la media divisa per country dell'attributo 'num_students' e si va a sostituirla laddove appaiono valori mancanti
media_studenti=times_max.groupby('country')['num_students'].mean()
media_studenti.head()


In [ ]:
i=0
for elem in times_max['num_students']:
    country=times_max['country'][i]
    media=media_studenti[country]
    #utilizzo della funzione .fillna per "riempire" i record aventi valore nullo 
    times_max['num_students'].fillna(media, inplace=True) 
    i+=1
            
times_max
            

In [ ]:
# utilizzo della funzione .str.split per la separazione della stringa riguardante l'attributo male_female_ratio
times_max[['female', 'male']] = times_max['female_male_ratio'].str.split('\:', expand=True) 

# viene convertito il tipo di tutti i valori dell'attributo female e male da str a float escludendo da questa operazione
# i valori mancanti (per entrambi gli attributi) e quelli contenenti il carattere '-' (per l'attributo 'female')
times_max['female']=times_max['female'][times_max['female']!='-'][times_max['female'].notnull()].astype('float')
times_max['male']=times_max['male'][times_max['male'].notnull()].astype('float')

#medie percentuali di femmine e maschi per ogni nazione
media_femmine=times_max.groupby('country')['female'].mean()
media_maschi=times_max.groupby('country')['male'].mean()

i=0
for elem in times_max['female']:
    country=times_max['country'][i]
    media=media_femmine[country]
    times_max['female'].fillna(media, inplace=True)
    i+=1
    
j=0
for elem in times_max['male']:
    country=times_max['country'][j]
    media=media_maschi[country]
    times_max['male'].fillna(media, inplace=True)
    j+=1
            
times_max.head()

In [ ]:
#uso della funzione round per l'arrotondamento alla terza cifra decimale
times_max['female']=round(times_max['female'], 3)
times_max['male']=round(times_max['male'], 3)

#calcolo delle percentuali totali di femmine e maschi sul totale degli studenti
times_max['tot_female']=(times_max['female']*times_max['num_students'])/100
times_max['tot_male']=(times_max['male']*times_max['num_students'])/100

times_max.head()

## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [ ]:
#viene calcolato, per ogni riga, il rapporto tra female/male
times_max['female/male']=times_max['female']/times_max['male']
times_max['female/male']=round(times_max['female/male'],3)

times_max.head()

In [ ]:
#viene calcolato il rapporto medio: media female / media male
rapporto_medio = np.mean(times_max['female/male'][np.isfinite(times_max['female/male'])])
rapporto_medio

In [ ]:
#viene verificato se il rapporto female/male sia minore del rapporto medio,
# se questo si verifica, l'università in questione viene messa inserita in under_ratio
under_ratio = times_max[times_max['female/male'] < rapporto_medio]
under_ratio.reset_index(level=0,inplace=True)

under_ratio.head()

## 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).¶

In [ ]:
#numeratore = somma studenti punto 6 per nazione 
#denominatore = a tot studenti nella nazione

In [ ]:
#numeratore 

tot_studenti_under_ratio = under_ratio.groupby('country')['num_students'].sum() 
tot_studenti_under_ratio.head()

In [ ]:
#denominatore

tot_studenti=times_max.groupby('country')['num_students'].sum()
tot_studenti.head()

In [ ]:
#calcolo del rapporto 

rapporto = tot_studenti_under_ratio/tot_studenti
rapporto

## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [140]:
edu_att = pd.read_csv('C:/Users/Antonino Guidotto/Desktop/2020/educational_attainment_supplementary_data.csv')
edu_att.tail()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79054,Last Updated: 10/20/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
edu_att[edu_att['country_name'].isnull()==True]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
edu_att[edu_att['series_name'].isnull()==True]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79054,Last Updated: 10/20/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
#utilizzo della funzione .dropna per eliminare le righe con valori mancanti negli attributi indicati
edu_att = edu_att.dropna(subset=['country_name','series_name'])

In [144]:
edu_att.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [151]:
# utilizzo della funzione .melt per convertire le colonne rappresentanti i vari anni in valori di riga
melted_df = pd.melt(edu_att, id_vars=['country_name','series_name'], value_vars=['1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015'])

In [155]:
melted_df_dropped = melted_df.dropna()
melted_df_dropped.rename(columns = {'variable':'year'} , inplace= True)
melted_df_dropped

C:\Users\Antonino Guidotto\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54000
...,...,...,...,...
2054020,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.48356
2054021,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.88820
2054022,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000
2054023,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000


## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [ ]:
times.head()

In [ ]:
# viene fatto una groupby per l'attributo 'university_name' e grazie alla funzione .size
# troviamo il numero di volte in cui un'università appare nel ranking
times_ranking=times.groupby('university_name').size()

#convertiamo l'oggetto di tipo groupby in un dataframe pandas
times_ranking_df=pd.DataFrame(times_ranking)

times_ranking_df.rename(columns={0 : 'n_times'}, inplace=True)
times_ranking_df.head()

In [ ]:
shanghai.head()

In [ ]:
shanghai_ranking=shanghai.groupby('university_name').size()
shanghai_ranking_df=pd.DataFrame(shanghai_ranking)
shanghai_ranking_df.rename(columns={0 : 'n_shanghai'}, inplace=True)
shanghai_ranking_df.head()

In [ ]:
cwur.head()

In [ ]:
#scegliamo di convertire il nome della colonna 'institution' in modo da renderlo omogeneo in tutti e 3 i dataframe
cwur.rename(columns={'institution' : 'university_name'}, inplace=True)
cwur_ranking=cwur.groupby('university_name').size()
cwur_ranking_df=pd.DataFrame(cwur_ranking)
cwur_ranking_df.rename(columns={0 : 'n_cwur'}, inplace=True)
cwur_ranking_df.head()

In [ ]:
unione_uni=pd.merge(pd.merge(times_ranking_df, shanghai_ranking_df, on='university_name'), cwur_ranking_df, on='university_name')
unione_uni.head()

In [ ]:
#restituisce il numero di volte in cui un'università appare
unione_uni['somma']=unione_uni['n_times']+unione_uni['n_shanghai']+unione_uni['n_cwur']

In [ ]:
unione_uni.head()

## 11. In the times ranking, compute the number of times each university appears

In [ ]:
times_ranking_df.head()

## 12. Find the universities that appear at most twice in the times ranking

In [ ]:
times_ranking_df[times_ranking_df['n_times']<= 2].head()

## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year)

Anno 2012-2013: non dobbiamo modificare nulla 
Anno 2014-2015: arrivare massimo fino a 400. Fino al 100 valori singoli, 101-150, 151-200, 201-300, 301-400. (usiamo i range di shanghai). Aggreghiamo times e creiamo i range per cwur.

In [ ]:
#estrazione degli attributi utili allo svolgimento del punto
position_times=times[['world_rank', 'university_name','year']]
position_shanghai=shanghai[['world_rank', 'university_name','year']]
position_cwur=cwur[['world_rank', 'university_name','year']]

In [ ]:
#iteriamo su position_times, e andiamo a modificare i ranking in modo da renderli omogenei
for i in range(0,len(position_times)):
    if position_times['year'][i]==2014 or position_times['year'][i]==2015:
        if '-' not in position_times['world_rank'][i]:
            if 101<=int(position_times['world_rank'][i] )<=150:
                position_times['world_rank'][i]='101-150'
            elif 151<=int(position_times['world_rank'][i] )<=200:
                position_times['world_rank'][i]='151-200'
        else:
            if position_times['world_rank'][i][0]=='2':
                position_times['world_rank'][i]='201-300'
            elif position_times['world_rank'][i][0]=='3':
                position_times['world_rank'][i]='301-400'
                
  

In [ ]:
for i in range(0,len(position_cwur)):
    if position_cwur['year'][i]==2014 or position_cwur['year'][i]==2015:
        if 101<=position_cwur['world_rank'][i]<=150:
            position_cwur['world_rank'][i]='101-150'
        elif 151<=position_cwur['world_rank'][i]<=200:
            position_cwur['world_rank'][i]='151-200'
        elif 201<=position_cwur['world_rank'][i]<=300:
            position_cwur['world_rank'][i]='201-300'
        elif 301<=position_cwur['world_rank'][i]<=400:
            position_cwur['world_rank'][i]='301-400'
            

In [ ]:
position_uni=pd.merge(pd.merge(position_times,position_shanghai, on='university_name'), position_cwur, on ='university_name')
position_uni.rename(columns={'world_rank_x' : 'w_r_times','year_x':'year_times', 'world_rank_y': 'w_r_shanghai',
                                    'year_y':'year_shanghai', 'world_rank': 'w_r_cwur', 'year': 'year_cwur'}, inplace=True)
position_uni.head()

In [ ]:
#cambiamento ordine delle colonne
position_uni = position_uni[['university_name','w_r_times','year_times','w_r_shanghai', 'year_shanghai','w_r_cwur','year_cwur']]

In [ ]:
#confronto tra tutti gli anni e le posizioni nei ranking per tutte le università
same_rank =[]
for index, row in position_uni.iterrows():
    if position_uni['year_times'][index]==position_uni['year_shanghai'][index] & position_uni['year_shanghai'][index]==position_uni['year_cwur'][index]:
        if position_uni['w_r_times'][index]==position_uni['w_r_shanghai'][index] and position_uni['w_r_shanghai'][index]==str(position_uni['w_r_cwur'][index]):
            same_rank.append(row)


same_rank_df = pd.DataFrame(same_rank)
same_rank_df

